# Implementação do algoritimo completo, sem compromisso com  perfomance

## Imports

In [1]:
import os
import functools
import operator
import math
import random
import sys

import numpy as np

## Função match

![Match function](./img/match_function.png)

In [2]:
def cluster_matching_function(weight_matrix,
                              cluster_number,
                              element,
                              prototypes,
                              dissimilarity_matrices):
    """
        :params: weight_matrix: numpy array-like 
                    matriz K x P de pesos das matrizes de dissimilaridades por cluster
                cluster_number: int
                    Número do cluster em questão
                element: int
                    Índice do elemento (entre 0 e N-1)
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade

        :return: float

    """

    # Criando aliases compatíveis com as variáveis da fórmula
    k = cluster_number
    D = dissimilarity_matrices
    p = len(D)
    Gk = prototypes[k]
    l = weight_matrix

    dissimilarities_sum = np.array([dj[element, Gk].sum() for dj in D])

    return np.dot(l[k], dissimilarities_sum)

## Função objetivo

![Objetive function](./img/objective_function.png)

In [3]:
def objective_function(clusters_qtd,
                       elements_qtd,
                       adequacy_criterion,
                       m,
                       weight_matrix,
                       prototypes,
                       dissimilarity_matrices):
    """
        :params: clusters_qtd: int
                    Quantidade total de clusters
                elements_qtd: int
                    Quantidade de elementos da base de dados
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                m: int
                    Fator de ponderação do índice de adequação
                weight_matrix:
                     matriz K x P de pesos das matrizes de dissimilaridades por cluster
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade

        :return: float

    """

    u = np.power(adequacy_criterion, m) # Resolvendo a exponeciação de u de uma vez só
    l = weight_matrix
    D = dissimilarity_matrices
    K = clusters_qtd
    G = prototypes
    N = elements_qtd
    match = cluster_matching_function # Criando um alias para reduzir o nome da função de matching
  
    J = np.array([np.array([u[i, k] * match(l, k, i, G, D) for i in range(N)]).sum() 
          for k in range(K)])


    return J.sum()

## Protótipos

![Prototype function](./img/prototype_function.png)

In [4]:
def get_prototypes(elements_qtd,
                   q,
                   m,
                   s,
                   cluster_number,
                   adequacy_criterion,
                   dissimilarity_matrices,
                   weight_matrix):
    """
        :params:
                elements_qtd: int 
                    Quantidade de elementos da base de dados
                q: int
                    Quantidade de elementos protótipos
                m: int
                    Fator de ponderação do índice de adequação
                s: int
                    Fator de ponderação dos pesos das matrizes
                cluster_number: int
                    Quantidade total de clusters
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                weight_matrix: 
                     matriz K x P de pesos das matrizes de dissimilaridades por cluster

        :return: list

    """

    G = []
    k = cluster_number
    D = dissimilarity_matrices
    u = np.power(adequacy_criterion, m)
    l = np.power(weight_matrix, s)
    N = elements_qtd
    p = len(D)
    
    p_sums = np.empty((p, N))
    
    for j in range(p):
        p_sums[j, :] = (D[j] * l[k,j]).sum(axis=1) 
    
    p_sums = p_sums.sum(axis=0)
  
    p_sums_rotten = u[:, k].flatten() * p_sums
    return p_sums_rotten.argsort()[-q:][::-1]

## Protótipos Dummy

![Prototype function](./img/prototype_function.png)

In [5]:
def get_prototypes_dummy(elements_qtd,
                       q,
                       m,
                       s,
                       cluster_number,
                       adequacy_criterion,
                       dissimilarity_matrices,
                       weight_matrix):
    G = []
    k = cluster_number
    D = dissimilarity_matrices
    u = np.power(adequacy_criterion, m)
    wm = np.power(weight_matrix, s)
    N = elements_qtd
    p = len(D)
    
    while (len(G) != q):
        menor_soma = 999999
        menor_indice = None
        
        for h in range(N):
            if h in G:
                continue
            soma = 0
            for i in range(N):
                soma_p = sum([wm[k,j] * D[j][:,h] for j in range(p)])
                soma += u[i,k] * soma_p

            if soma < menor_soma:
                menor_soma = soma
                menor_indice = h
                
        G.append(menor_indice)
        
    return G

  
def get_prototypes_dummy2(elements_qtd,
                       q,
                       m,
                       s,
                       cluster_number,
                       adequacy_criterion,
                       dissimilarity_matrices,
                       weight_matrix):
    G = []
    k = cluster_number
    D = dissimilarity_matrices
    u = np.power(adequacy_criterion, m)
    l = np.power(weight_matrix, s)
    N = elements_qtd
    p = len(D)
    
    while (len(G) != q):
        menor_soma = 999999
        menor_indice = None
        
        for h in range(N): 
            if h in G:
                continue
                
            sums_p = np.array([D[j][:, h] * l[k,j] for j in range(p)]).sum(axis=0)
            soma = np.dot(u[:, k], sums_p)
            
            if soma < menor_soma:
                menor_soma = soma
                menor_indice = h
                 
        G.append(menor_indice)
        
    return G
            

a = get_prototypes(elements_qtd = 3,
                   q=2,
                   m=1.6,
                   s=1,
                   cluster_number=0,
                   adequacy_criterion = np.array([[1,2,3],[3,2,1],[0,1,3]]),
                   dissimilarity_matrices= [np.array([[0, .5, .1],[.3, 0, .1], [1, .2, 0]])],
                   weight_matrix=np.array([[0.8],[0.5]]))

b = get_prototypes_dummy2(elements_qtd = 3,
                   q=2,
                   m=1.6,
                   s=1,
                   cluster_number=0,
                   adequacy_criterion = np.array([[1,2,3],[3,2,1],[0,1,3]]),
                   dissimilarity_matrices= [np.array([[0, .5, .1],[.3, 0, .1], [1, .2, 0]])],
                   weight_matrix=np.array([[0.8],[0.5]]))
print(a, b)

[1 0] [1, 2]


## Matriz de relevâcia

![Funções de peso](./img/vector_weights_function.png)

In [6]:
def compute_relevance_weights(clusters_qtd,
                              dissimilarity_matrices,
                              prototypes,
                              elements_qtd,
                              adequacy_criterion,
                              m):
    """
        :params:
                clusters_qtd: int
                    Quantidade total de clusters
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                elements_qtd: int
                    Quantidade de elementos da base de dados
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                m: int
                    Fator de ponderação do índice de adequação

        :return: numpy array of shape K x P

    """

    D = dissimilarity_matrices
    P = len(D)
    Gk = prototypes
    K = clusters_qtd
    N = elements_qtd
    u = np.power(adequacy_criterion, m)
    l = np.zeros((K, P))

    def match(element, Dh, G):
        """
            Função auxiliar para cálculo de match entre um elemento 
            qualquer, os protótipos G de um cluster específico e uma matriz 
            de similaridade específica Dh.
        """

        return Dh[element, G].sum()

    for k in range(K):
        # Calculado o somatório do numerador da equação à esquerda da igualdade
        weight_diss_sum1 = np.array([np.array([u[i, k] * match(i, D[h], Gk[k]) for i in range(N)]).sum()
                            for h in range(P)])
        for j in range(P):
     
            # Calculado o somatório do denominador da equação à esquerda da igualdade
#             weight_diss_sum2 = np.array([u[i, k] * match(i, D[j], Gk[k])
#                                     for i in range(N)]).sum()
            
            weight_diss_sum2 = weight_diss_sum1[j]

            # Executando a divisão da fração à esquerda da equação
            l[k, j] = math.pow(weight_diss_sum1.prod(), 1/P) / weight_diss_sum2

    return l

## Grau de pertinência

![Fórmula grau de pertinência](./img/membership_degree.png)

In [7]:
def compute_membership_degree(weight_matrix,
                              prototypes,
                              clusters_qtd,
                              dissimilarity_matrices,
                              elements_qtd,
                              m):
    """
        :params: weight_matrix: numpy array-like 
                    matriz K x P de pesos das matrizes de dissimilaridades por cluster
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                clusters_qtd: int
                    Quantidade total de clusters
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                elements_qtd: int
                    Quantidade de elementos da base de dados
                m: int
                    Fator de ponderação do índice de adequação

        :return: numpy array NxK

    """
        

    K = clusters_qtd
    G = prototypes
    D = dissimilarity_matrices
    l = weight_matrix
    P = len(D)
    N = elements_qtd
    u = np.zeros((N, K))
    
    match = cluster_matching_function # Criando um alias para reduzir o nome da função de matching

    def ratio(element, k, h):
        r = match(l, k, element, G, D) / match(l, h, element, G, D)
        return math.pow(r, 1/(m-1))

    for i in range(N):
        for k in range(K):
            outter_sum = np.array([ratio(i, k, h) for h in range(K)]).sum()
            u[i, k] = 1/outter_sum

    return u

## Carregando Matrizes

In [8]:
def carregar_matrizes(data_path = None):
    data_path = data_path or "./data"
    data_path = os.path.abspath(data_path)
    
    FAC_FILE = os.path.join(data_path, "mfeat-fac-dissimilarity.npy")
    FOU_FILE = os.path.join(data_path, "mfeat-fou-dissimilarity.npy")
    KAR_FILE = os.path.join(data_path, "mfeat-kar-dissimilarity.npy")

    fac_dis = np.load(FAC_FILE)
    fou_dis = np.load(FOU_FILE)
    kar_dis = np.load(KAR_FILE)
    
    return fac_dis, fou_dis, kar_dis

data_path = sys.argv[1] if len(sys.argv) == 2 else None
fac_dis, fou_dis, kar_dis = carregar_matrizes(data_path)

## Algoritmo completo
> Partitioning fuzzy K-medoids clustering algorithms with relevance weight for each dissimilarity matrix estimated locally

* Parametros: $K = 10; m = 1.6; T = 150; \epsilon = 10^{−10};$
* Devemos considerar a iniciarlizar do vetor de pesos como sendo 1, já que usamos a equação 9 (MFCMdd-RWL-P)

In [9]:
def random_prototypes(K, N, q, seed):
    elements = set(range(N))
    protos = []
    random.seed(seed)
    for k in range(K):
        protos.append(random.sample(elements, q))
        elements -= set(protos[-1])

    return protos

def assert_relevance_weights_prod_one(l):
    # l.shape == (K,P)
    prods_p = l.prod(axis=1)
    assert round(prods_p.sum()) == l.shape[0], f"O produto dos pesos de relevância não é igual a 1 ({prods_p.sum()})"

def assert_membership_degree_sum_one(u):
     # u.shape == (N,K)
    sums_k = u.sum(axis=1)
    assert round(sums_k.sum()) == u.shape[0], f"A soma dos pesos de relevância não é igual a 1 ({sums_k.sum()})"

def executar_treinamento(dissimilarity_matrices,
                       elements_qtd,
                       K=10,
                       m=1.6,
                       T=150,
                       epsilon=10e-10,
                       q=3, 
                       seed=13082020):

    D = dissimilarity_matrices
    N = elements_qtd
    P = len(D)

    last_lambda = np.ones((K, P))
    last_prototypes = random_prototypes(K, N, q, seed)
    last_membership_degree = None
    last_cost = None
    
    assert_relevance_weights_prod_one(last_lambda)

#     print("Passo 0")
#     print("Calculando matriz de adequação inicial (u0)")
    u0 = compute_membership_degree(weight_matrix=last_lambda,
                                   prototypes=last_prototypes,
                                   clusters_qtd=K,
                                   dissimilarity_matrices=dissimilarity_matrices,
                                   elements_qtd=N,
                                   m=m)
    
#     assert_membership_degree_sum_one(u0)

#     print("Calculando função de custo inicial (J0)")
    J0 = objective_function(clusters_qtd=K,
                            elements_qtd=N,
                            adequacy_criterion=u0,
                            m=m,
                            weight_matrix=last_lambda,
                            prototypes=last_prototypes,
                            dissimilarity_matrices=dissimilarity_matrices)
    
    last_membership_degree = u0
    last_cost = J0
    
    for t in range(1, T):
#         print(f"Passo {t}/{T}")
        
#         print(">> Calculando protótipos")
        new_prototypes = [get_prototypes_dummy2(elements_qtd=N,
                                         q=q,
                                         m=m,
                                         s=1,
                                         cluster_number=k,
                                         adequacy_criterion=last_membership_degree,
                                         dissimilarity_matrices=D,
                                         weight_matrix=last_lambda) for k in range(K)]
        
        #print("new_prototypes.shape", new_prototypes)
        
#         print(">> Calculando matriz de relevâncias")
        new_lambda = compute_relevance_weights(clusters_qtd=K,
                                               dissimilarity_matrices=D,
                                               prototypes=new_prototypes,
                                               elements_qtd=N,
                                               adequacy_criterion=last_membership_degree,
                                               m=m)
        
#         assert_relevance_weights_prod_one(new_lambda)
    
#         print(">> Calculando grau de pertinência")
        new_degree = compute_membership_degree(weight_matrix=new_lambda,
                                               prototypes=new_prototypes,
                                               clusters_qtd=K,
                                               dissimilarity_matrices=dissimilarity_matrices,
                                               elements_qtd=N,
                                               m=m)
    
        
#         assert_membership_degree_sum_one(new_degree)

#         print(">> Calculando função objetivo")
        new_cost = objective_function(clusters_qtd=K,
                                      elements_qtd=N,
                                      adequacy_criterion=new_degree,
                                      m=m,
                                      weight_matrix=new_lambda,
                                      prototypes=new_prototypes,
                                      dissimilarity_matrices=dissimilarity_matrices)

        last_prototypes = new_prototypes
        last_lambda = new_lambda
        last_membership_degree = new_degree
#         print(">> Cost: ", new_cost)
        
        if abs(last_cost - new_cost) <= epsilon:
            last_cost = new_cost
            break
    
        last_cost = new_cost
        
    data = {
        "cost":last_cost,
        "membership_degree":last_membership_degree,
        "prototypes":last_prototypes,
        "weight_matrix":last_lambda,
        "times": t,
        "q": q,
        "K":K,
        "m":m,
        "seed": seed,
    }
    return data

## Executando 100x

In [10]:
import multiprocessing as mp

def executar_algoritmo(dissimilarity_matrices, N, times=100, q=2):

    best = None
    
    seeds = [18082020 + i for i in range(times)]
    with mp.Pool() as p:
        results = []
        for seed in seeds:
            kwds = dict(q=q, seed = seed)
            r = p.apply_async(executar_treinamento, (dissimilarity_matrices, N), kwds)
            results.append(r)
            
        for i, r in enumerate(results):
            data = r.get()
            print(f"Execução {i+1}/{times}")
            print(">> Cost: ", data["cost"] )
            if (not best) or data["cost"] < best["cost"]:
                best = data 
        
            
    return best
print('>> FAC')
melhor_resultado_fac = executar_algoritmo([fac_dis], 2000, times=100)
print('>> FOU')
melhor_resultado_fou = executar_algoritmo([fou_dis], 2000, times=100)
print('>> KAR')
melhor_resultado_kar = executar_algoritmo([kar_dis], 2000,times=100)
print('>> TODAS')
melhor_resultado_todas = executar_algoritmo([fac_dis, fou_dis, kar_dis], 2000, times=100)


>> FAC
Execução 1/100
>> Cost:  3383.234988756842
Execução 2/100
>> Cost:  3387.2842090236727
Execução 3/100
>> Cost:  3378.3481717937234
Execução 4/100
>> Cost:  3380.0182245201595
Execução 5/100
>> Cost:  3388.150356620451
Execução 6/100
>> Cost:  3381.290668091048
Execução 7/100
>> Cost:  3386.230303478495
Execução 8/100
>> Cost:  3397.3193919374944
Execução 9/100
>> Cost:  3390.9564695919244
Execução 10/100
>> Cost:  3387.1007221443074
Execução 11/100
>> Cost:  3380.0317722874815
Execução 12/100
>> Cost:  3393.574539716901
Execução 13/100
>> Cost:  3384.36287296341
Execução 14/100
>> Cost:  3385.0839343660655
Execução 15/100
>> Cost:  3386.8424551358685
Execução 16/100
>> Cost:  3398.94034815121
Execução 17/100
>> Cost:  3386.044237725643
Execução 18/100
>> Cost:  3383.963117638917
Execução 19/100
>> Cost:  3380.001516908644
Execução 20/100
>> Cost:  3380.675316940849
Execução 21/100
>> Cost:  3396.779345989508
Execução 22/100
>> Cost:  3391.6119506406385
Execução 23/100
>> Cost:  

Execução 85/100
>> Cost:  1799.9911159899789
Execução 86/100
>> Cost:  1800.3102221554177
Execução 87/100
>> Cost:  1800.1410798003044
Execução 88/100
>> Cost:  1800.333436512324
Execução 89/100
>> Cost:  1800.7480424497012
Execução 90/100
>> Cost:  1805.4690895691792
Execução 91/100
>> Cost:  1801.1027539130746
Execução 92/100
>> Cost:  1800.748076170606
Execução 93/100
>> Cost:  1800.6268037744715
Execução 94/100
>> Cost:  1800.6968268509727
Execução 95/100
>> Cost:  1800.3334365123237
Execução 96/100
>> Cost:  1800.56102953018
Execução 97/100
>> Cost:  1800.270633230421
Execução 98/100
>> Cost:  1803.8270996469073
Execução 99/100
>> Cost:  1800.6475898999188
Execução 100/100
>> Cost:  1800.4341935576729
>> KAR
Execução 1/100
>> Cost:  1517.5510747247818
Execução 2/100
>> Cost:  1519.057084931343
Execução 3/100
>> Cost:  1518.7894550521423
Execução 4/100
>> Cost:  1518.1125548001228
Execução 5/100
>> Cost:  1518.753650124013
Execução 6/100
>> Cost:  1518.6922023043767
Execução 7/100


Execução 70/100
>> Cost:  6644.103917068699
Execução 71/100
>> Cost:  6655.141084939526
Execução 72/100
>> Cost:  6655.165904636039
Execução 73/100
>> Cost:  6639.145224537437
Execução 74/100
>> Cost:  6662.770144798655
Execução 75/100
>> Cost:  6648.673023777358
Execução 76/100
>> Cost:  6670.091030679252
Execução 77/100
>> Cost:  6650.7170444666845
Execução 78/100
>> Cost:  6658.442334290456
Execução 79/100
>> Cost:  6675.367505190616
Execução 80/100
>> Cost:  6660.996483018851
Execução 81/100
>> Cost:  6665.540097755159
Execução 82/100
>> Cost:  6650.840855701905
Execução 83/100
>> Cost:  6668.179685218998
Execução 84/100
>> Cost:  6655.244787152007
Execução 85/100
>> Cost:  6649.866106546768
Execução 86/100
>> Cost:  6667.555243031788
Execução 87/100
>> Cost:  6659.427283444576
Execução 88/100
>> Cost:  6648.988130527322
Execução 89/100
>> Cost:  6665.337425053363
Execução 90/100
>> Cost:  6663.661863996207
Execução 91/100
>> Cost:  6655.21658978627
Execução 92/100
>> Cost:  6661.0

### Melhores resultados

In [11]:
print("Melhor custo pro fac", melhor_resultado_fac["cost"])
print("Melhor custo pro fou", melhor_resultado_fou["cost"])
print("Melhor custo pro kar", melhor_resultado_kar["cost"])
print("Melhor custo com as três matrizes", melhor_resultado_todas["cost"])

Melhor custo pro fac 3376.6806758686553
Melhor custo pro fou 1799.8044198166776
Melhor custo pro kar 1516.7965475374451
Melhor custo com as três matrizes 6639.145224537437


### Exportando melhores resultados

In [12]:
import pickle
 
def export_best_result(data, file_name):
    with open(file_name, "wb") as f:
        pickle.dump(data, f)
   
export_best_result(melhor_resultado_fac, "data/melhor_resultado_fac.pickle")
export_best_result(melhor_resultado_fou, "data/melhor_resultado_fou.pickle")
export_best_result(melhor_resultado_kar, "data/melhor_resultado_kar.pickle")
export_best_result(melhor_resultado_todas, "data/melhor_resultado_todas.pickle")

### Exportando paricões fuzzy

In [13]:
import pandas as pd

def export_fuzzy_partitions_to_csv(data, file_name):
    df = pd.DataFrame(data["membership_degree"])
    df.to_csv(file_name, index=False, decimal=',')

export_fuzzy_partitions_to_csv(melhor_resultado_fac, "data/fuzzy_partitions_fac.csv")
export_fuzzy_partitions_to_csv(melhor_resultado_fou, "data/fuzzy_partitions_fou.csv")
export_fuzzy_partitions_to_csv(melhor_resultado_kar, "data/fuzzy_partitions_kar.csv")
export_fuzzy_partitions_to_csv(melhor_resultado_todas, "data/fuzzy_partitions_todas.csv")

## Partições hard

In [24]:
def get_hard_patitions(membership_degree):
    """
        membership_degree: numpy array of shape N x K
    """
    u = membership_degree
    members = []
    K = membership_degree.shape[1]
    
    # Obtendo o índice do grupo em que cada elemento possui maior valor de pertencimento
    element_index_membership = u.argsort(axis=1)[:, -1]
    print(u.argsort(axis=1))
    
    pd.DataFrame(u.argsort(axis=1)).to_csv("data/membership_degree.csv")
    
    for k in range(K):
        # Para cada grupo k, extrai quais dos elementos possui maior grau de pertencimento para ele
        memb = np.where(element_index_membership == k)[0]
#         print(np.where(element_index_membership == k), k)
        members.append(memb)
        
    return members

membership_degree = melhor_resultado_todas["membership_degree"]
hard_members = get_hard_patitions(membership_degree)

for group_number, group in enumerate(hard_members):
    print(group_number, group, len(group))

print("Total de elementos somados:",  sum([len(g) for g in hard_members]))

[[3 5 6 ... 4 0 7]
 [1 8 6 ... 4 0 7]
 [3 1 8 ... 4 0 7]
 ...
 [7 2 4 ... 6 5 3]
 [7 0 4 ... 6 5 3]
 [7 0 4 ... 8 1 3]]
0 [ 421  430  480  501  514  534  550  552  558  776  829  855  885  901
  908  919  942  948  956  961  970  988 1003 1065 1091 1102 1105 1113
 1121 1124 1128 1197 1201 1202 1203 1204 1208 1212 1213 1216 1219 1221
 1224 1225 1227 1228 1231 1232 1236 1238 1244 1248 1252 1254 1255 1258
 1260 1261 1262 1273 1283 1285 1286 1291 1292 1312 1313 1319 1325 1326
 1328 1331 1333 1336 1339 1340 1341 1343 1346 1347 1351 1352 1353 1354
 1355 1360 1363 1368 1376 1377 1381 1383 1384 1385 1393 1399 1600 1601
 1602 1605 1607 1608 1611 1612 1613 1614 1615 1616 1617 1619 1620 1621
 1622 1626 1627 1628 1629 1631 1633 1634 1635 1636 1637 1638 1640 1641
 1643 1644 1645 1647 1648 1650 1651 1652 1653 1654 1655 1656 1657 1658
 1659 1660 1661 1662 1663 1664 1668 1670 1671 1672 1673 1674 1675 1676
 1677 1681 1682 1683 1685 1686 1689 1690 1691 1692 1693 1695 1697 1698
 1700 1701 1702 1703 1706 

## Funções de validação do cluster

In [15]:
def calc_partition_coefficient(membership_degree):
    n = membership_degree.shape[0]
    membership_degree = np.power(membership_degree, 2)
    return membership_degree.sum(axis = 1).sum()/n

def calc_modified_partition_coefficient(membership_degree):
    c = membership_degree.shape[1]
    vpc = calc_partition_coefficient(membership_degree)
    return 1 - (c/(c-1))*(1-vpc)

def calc_partition_entropy(membership_degree):
    n = membership_degree.shape[0]
    membership_degree = np.log10(membership_degree) * membership_degree
    return -membership_degree.sum(axis = 1).sum()/n

partition_coeficient = calc_modified_partition_coefficient(membership_degree)
partition_entropy = calc_partition_entropy(membership_degree)

print("Modified partition coefficient:", partition_coeficient)
print("Parition Entropy:", partition_entropy)


Modified partition coefficient: 0.0048603134184753705
Parition Entropy: 0.9915984356709501
